![alt text](https://zewailcity.edu.eg/main/images/logo3.png)
# Math 404 report 1
### Ezzat Esam Eisawy  201901195

#### Imports

In [22]:
import numpy as np
from numpy import array
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable


#### Functions

In [23]:
def _simplex_iterate(a : np.array ,c : np.array , basic_vars : list , nbasic_vars : list , B : np.array):
    """ The simplex iteration function. 
    updates the basic and non basic variables, used for both phases
    INTERNAL USE ONLY
    """    

    iterations = 0
    while 1 :

        b = a[:,basic_vars]    # basis
        Cb = c[basic_vars]     

        b_inv = np.linalg.inv(b)   # normal inversion, can be optimized
        Xb = b_inv@B

        z_c =  Cb@b_inv@a[:,nbasic_vars] - c[nbasic_vars]    

        if any(z_c<0):   # optimality condition not met
            entering_var_idx = np.argmin(z_c)

        else : 
            break
        y_entering_var = b_inv@a[:,nbasic_vars[entering_var_idx]]
        

        with np.errstate(divide='ignore'):   # ignore divide by zero message
            ratio = (Xb / y_entering_var)
        
        non_negative_idcs = np.where(y_entering_var > 0)[0]

        if len(non_negative_idcs) > 0:
            leaving_var_idx =  non_negative_idcs[np.argmin(ratio[non_negative_idcs])]

        else:                   # no positive ratios
            return None         # failure
        

        # swap leaving basic and entering non basic variables 
        basic_vars[leaving_var_idx] , nbasic_vars[entering_var_idx] = nbasic_vars[entering_var_idx] , basic_vars[leaving_var_idx]

        iterations += 1

    return basic_vars , nbasic_vars , iterations 


def lin_prog(C :np.array , A : np.array = []  ,    B : np.array = []  ,E : np.array = [],  B_e : np.array = [] ,minimize = False ) -> dict :
    """ Solves a LLP problem in the form 
    Max Cx,
    subject to
    Ax <= b
    Ex = B_e
    x >= 0

    Args:
        C (np.array): Coefficients of the objective function
        A (np.array, optional): Coefficients of the inequality constraints
        E (np.array, optional): Coefficients of the equality constraints. Defaults to None.
        B (np.array, optional): Right-hand side of the inequality constraints
        B_e (np.array, optional): Right-hand side of the equality constraints. Defaults to None.
    Returns:
        dict: Solution of the problem with "status" key. If the status is "SUCCESS", it also contains the following keys:
            - optimal_value (float): Optimal value of the objective function
            - variables (dict): Dictionary of variables and their values
            - iterations (int): Number of iterations performed
    """    


    # def expand_constraints(a : np.array) :
    #     a = np.hstack((a, np.diag(np.ones(len_slacks))))
    #     c = np.hstack((C,np.zeros(len_slacks)))
    #     return a , c


    iterations = 0
    if minimize :
        C *= -1
    if not  len(E)  and len(B) and  np.all(B > 0) :    # single phase
        len_slacks = len(A)
        
        a , c  = np.hstack((A, np.diag(np.ones(len_slacks)))) , np.hstack((C,np.zeros(len_slacks)))
        len_xs = len(C)
        basic_vars = [i for i in range(len_xs , len_xs  + len_slacks )]
        nbasic_vars = [i for i in range(len_xs)]

    else :    # two phase 

        len_slacks = len(A)
        len_xs = len(C)
        len_equalities = len(E)

        artificial_vars_locs = []
        diag = []
        for i in range(len_slacks):
            if B[i] < 0:
                B[i] *=-1
                A[i] *= -1
                artificial_vars_locs.append(i)
                diag.append(-1)
            else :
                diag.append(1)

        for i in range(len(E)) :
            if B_e[i] < 0:
                B_e[i] *=-1
                E[i] *= -1
            artificial_vars_locs.append(i + len_slacks)
            
            

        len_avs = len(artificial_vars_locs)
        artificial_vars = np.zeros(shape = (len_slacks + len_equalities ,len(artificial_vars_locs)) )
        
        # create the Avs matrix to be concatenated with the constraints
        for i , idx in enumerate(artificial_vars_locs):
            artificial_vars[idx , i] = 1


        if len(E) and len(A) :   # both inequality and equality
            padded_zeros = np.zeros(shape = ( len_equalities , len_slacks))
            a_temp  = np.hstack(( np.vstack((A,E))      , np.vstack((np.diag(diag),padded_zeros)) , )) 
            B = np.hstack((B,B_e))
        elif len(E) :            # only equality
            a_temp  = E 
            B = B_e
        elif len(A) :            # only inequality
            a_temp   = np.hstack((A, np.diag(diag))) 
        else :
            raise ValueError("No constraints")
        
        a = np.hstack((a_temp,artificial_vars))
        
        c_old_fun = np.hstack((C,np.zeros(len_slacks)))
        c = np.hstack((np.zeros(len_slacks + len_xs ) ,-np.ones(len_avs)))
        all_vars = [i for i in range(len_slacks + len_xs + len_avs)] 

        basic_vars = []
        nbasic_vars = []
        
        # get the basic variables
        for i , var in enumerate(all_vars[:-len(artificial_vars_locs)]):
            if i >= len_xs and  diag[i - len_xs] == 1:
                basic_vars.append(var)
            else :
                nbasic_vars.append(var)
        basic_vars += [i for i in range(len_slacks + len_xs , len_slacks + len_xs + len_avs)]


        results = _simplex_iterate(a ,c , basic_vars , nbasic_vars , B)

        if results  is not None:
            basic_vars , nbasic_vars , iterations = results

            # to compute the value of z
            b = a[:,basic_vars]
            Cb = c[basic_vars]
            b_inv = np.linalg.inv(b)

            Xb = b_inv@B
            z_val = Cb@b_inv@B

            
            if z_val != 0:   # must be 0
                return {
                    "status" : "FAILURE",
                    "message" : "no feasible starting point"
                }

            # remove avs from A
            a = a[:,:-len_avs]

            # remove avs from non basic vars
            nbasic_vars_temp = nbasic_vars.copy()
            for val in nbasic_vars_temp:
                if val in list(range(len_slacks + len_xs , len_slacks + len_xs+ len_avs)):
                    nbasic_vars.remove(val)
                
            
            # reset to old function
            c = c_old_fun
            
            
        else :
            return {
                "status" : "FAILURE",
                "message" : "no feasible starting point"
            }

        
   
    # the traditional phase step
    results = _simplex_iterate(a ,c , basic_vars , nbasic_vars , B)

    if results is not None:
        basic_vars , nbasic_vars , iterations2 = results
        b = a[:,basic_vars]
        Cb = c[basic_vars]
        b_inv = np.linalg.inv(b)
        Xb = b_inv@B
        z_val = Cb@b_inv@B
        if minimize:    
            z_val *= -1
        values = {}
        for i , v in enumerate(basic_vars):
            values[f"x{v+1}"] = Xb[i]
        for i , v in enumerate(nbasic_vars):
            values[f"x{v+1}"] = 0
        values = dict(sorted(values.items()))

        

        return {
            "status" : "SUCCESS",
            "iterations" : iterations + iterations2,
            "variables" : values,
            "optimal_value" : z_val
        }
    
    return {
        "status" : "FAILURE",
        "message" : "unbounded"
    }
    

#### Examples

#### Example 1
**Objective Function:**
$$ \text{Maximize } Z = 2x_1 + x_2 $$

**Constraints:**
$$ 3x_1 + 4x_2 \leq 6 $$
$$ 6x_1 + x_2 \leq 3 $$

**Non-negativity Constraints:**
$$ x_1 \geq 0 $$
$$ x_2 \geq 0 $$


##### Using my implementation

In [24]:
C = array([2,1])
A = array([[3,4],[6,1]])
b = array([6,3])

lin_prog(C,A,b)

{'status': 'SUCCESS',
 'iterations': 2,
 'variables': {'x1': 0.2857142857142857,
  'x2': 1.2857142857142856,
  'x3': 0,
  'x4': 0},
 'optimal_value': 1.857142857142857}

##### Using pulp

In [25]:
# Create a linear programming problem
prob = LpProblem("Maximize_Z", LpMaximize)

# Define decision variables
x1 = LpVariable("x1", lowBound=0)  # x1 >= 0
x2 = LpVariable("x2", lowBound=0)  # x2 >= 0

# Define the objective function
prob += 2 * x1 + x2, "Z"

# Define constraints
prob += 3 * x1 + 4 * x2 <= 6, "Constraint1"
prob += 6 * x1 + x2 <= 3, "Constraint2"

# Solve the problem
prob.solve()

# Display the results
print("Status:", prob.status)
print("Optimal values:")
print("x1 =", x1.value())
print("x2 =", x2.value())
print("Optimal objective function value (Z) =", prob.objective.value())


Status: 1
Optimal values:
x1 = 0.28571429
x2 = 1.2857143
Optimal objective function value (Z) = 1.85714288


#### Example 2
**Objective Function:**
$$ \text{Minimize } Z = 2x_1 + 4x_2 $$

**Constraints:**
$$ x_1 + x_2 \leq 8 $$
$$ 6x_1 + 4x_2 \geq 12 $$
$$ x_1 + 4x_2 = 20 $$

**Non-negativity Constraints:**
$$ x_1 \geq 0 $$
$$ x_2 \geq 0 $$


##### Using my implementation

In [26]:
C = array([2,4])

A = array([
    [1,1],
    [-6,-4]
])

B = array([8,-12])

E = array([[1,4]])

B_e = array([20])

lin_prog(C , A , B, E , B_e , minimize= True)

{'status': 'SUCCESS',
 'iterations': 2,
 'variables': {'x1': 0, 'x2': 5.0, 'x3': 3.0, 'x4': 8.0},
 'optimal_value': 20.0}

##### Using pulp

In [27]:
# Create a linear programming problem
prob = LpProblem("Minimize_Z", LpMinimize)

# Define decision variables
x1 = LpVariable("x1", lowBound=0)  # x1 >= 0
x2 = LpVariable("x2", lowBound=0)  # x2 >= 0

# Define the objective function
prob += 2 * x1 + 4 * x2, "Z"

# Define constraints
prob += x1 + x2 <= 8, "Constraint1"
prob += 6 * x1 + 4 * x2 >= 12, "Constraint2"
prob += x1 + 4 * x2 == 20, "Constraint3"

# Solve the problem
prob.solve()

# Display the results
print("Status:", prob.status)
print("Optimal values:")
print("x1 =", x1.value())
print("x2 =", x2.value())
print("Optimal objective function value (Z) =", prob.objective.value())

Status: 1
Optimal values:
x1 = 0.0
x2 = 5.0
Optimal objective function value (Z) = 20.0


#### Example 3

**Objective Function:**
$$ \text{Maximize } Z = x_1 - 8x_2 $$

**Constraints:**
$$ 3x_1 + 2x_2 \geq 6 $$
$$ x_1 - x_2 \leq 6 $$
$$ 9x_1 + 7x_2 \leq 108 $$
$$ 3x_1 + 7x_2 \leq 70 $$
$$ -2x_1 + 5x_2 \leq 35 $$


**Non-negativity Constraints:**
$$ x_1 \geq 0 $$
$$ x_2 \geq 0 $$


##### Using my implementation

In [28]:
C = np.array([1, -8])

A = np.array([[-3, -2],
              [1, -1],
              [9, 7],
              [3, 7],
              [-2, 5]])

B = np.array([-6, 6, 108, 70, 35])

lin_prog(C , A , B)

{'status': 'SUCCESS',
 'iterations': 2,
 'variables': {'x1': 6.0,
  'x2': 0,
  'x3': 12.0,
  'x4': 0,
  'x5': 54.0,
  'x6': 52.0,
  'x7': 47.0},
 'optimal_value': 6.0}

##### Using pulp

In [29]:
x = LpVariable("x1", lowBound=0)
y = LpVariable("x2", lowBound=0)

problem = LpProblem("propblem", sense= LpMaximize)

problem += (x - 8*y) # maximize function
problem += (3*x + 2*y >= 6  ) # constraint 1
problem += (x -y <= 6  ) # constraint 2
problem += (9*x +7*y <= 108) # constraint 3
problem += (3*x +7 *y <= 70) # constraint 4
problem += (-2*x +5*y <= 35 ) # constraint 5

problem.solve()
for variable in problem.variables():
    print(f"{variable.name} = {variable.value()}")

print(f"objective = {problem.objective.value()}")

x1 = 6.0
x2 = 0.0
objective = 6.0


#### Example 4
**Objective Function:**
$$ \text{Minimize } Z = 3x_1 + 2x_2 + x_3 $$

**Constraints:**
$$ x_1 + x_2 + x_3 \geq 4 $$
$$ x_2 - x_3 \leq 2 $$
$$ x_1 +x_2 +2x_3 = 6 $$


**Non-negativity Constraints:**
$$ x_1 \geq 0 $$
$$ x_2 \geq 0 $$
$$ x_3 \geq 0 $$

In [30]:
C = np.array([3,2,1])

A = np.array([
    [-1,-1,-1],
    [0,1,-1]
])
B = np.array([-4 ,2])

E = np.array([
    [1,1,2],
])
B_e = np.array([6])

lin_prog(C , A , B, E , B_e , minimize= True)

{'status': 'SUCCESS',
 'iterations': 3,
 'variables': {'x1': 0, 'x2': 2.0, 'x3': 2.0, 'x4': 0, 'x5': 2.0},
 'optimal_value': 6.0}

##### Using pulp

In [31]:
x1 = LpVariable("x1", lowBound=0)
x2 = LpVariable("x2", lowBound=0)
x3 = LpVariable("x3", lowBound=0)

problem = LpProblem("propblem", sense= LpMinimize)

problem += (3*x1 + 2*x2 + x3) # maximize function
problem += (x1 + x2 + x3 >= 4  ) # constraint 1
problem += (x2 - x3 <= 2  ) # constraint 2
problem += (x1 + x2 + 2 *x3 ==6) # constraint 3

problem.solve()
for variable in problem.variables():
    print(f"{variable.name} = {variable.value()}")

print(f"objective = {problem.objective.value()}")

x1 = 0.0
x2 = 2.0
x3 = 2.0
objective = 6.0
